In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime

## Group Assignment
### Team Number: 16
### Team Member Names: Matthew Gong, Arya Patel, Saihej Singh
### Team Strategy Chosen: __________(RISKY OR SAFE)

In [6]:
possibleTickers = pd.read_csv("Tickers_Example.csv")
tickers = []
tickers_hist = {}
minVolume = 150000
allowed_currencies = ["USD", "CAD"]
start_date = "2023-01-01"
end_date = "2023-10-31"
threshold_day = 18
days_in_month = []

for ticker in possibleTickers[possibleTickers.columns.values[0]]:
    currTicker = yf.Ticker(ticker)
    currHist = currTicker.history(start=start_date, end=end_date)
    currFastInfo = currTicker.fast_info

    if currHist.empty:
        print(ticker, "is delisted.")
    elif currTicker in tickers:
        print(ticker, "is a duplicate.")
    else:
        tickers_hist[ticker] = currHist

        group_months = currHist.reset_index().groupby(pd.Grouper(key='Date', freq='M'))
        dfs = [group for _, group in group_months]
        filtered_dfs = []
        for i, df in enumerate(dfs):
            #recombine all the months with more trading days than set threshold
            if len(df.index) >= threshold_day:
                days_in_month.append(len(df.index))
                filtered_dfs.append(df)
        currHist_updated = pd.concat(filtered_dfs)
        currHist_updated.set_index("Date", inplace=True)

        if currFastInfo["currency"] not in allowed_currencies:
            print(ticker, "is not in USD or CAD")
        elif currHist["Volume"].mean() * (np.mean(days_in_month)) < minVolume:
            print(ticker, "only has a volume of", currFastInfo["lastVolume"], "and average days of", np.mean(days_in_month))
        else:
            tickers.append(ticker)

print(possibleTickers.values)

AGN: No timezone found, symbol may be delisted


AGN is delisted.


CELG: No timezone found, symbol may be delisted


CELG is delisted.


MON: No timezone found, symbol may be delisted


MON is delisted.


RTN: No timezone found, symbol may be delisted


RTN is delisted.
[['ABBV']
 ['ABT']
 ['ACN']
 ['AGN']
 ['AIG']
 ['AMZN']
 ['AXP']
 ['BA']
 ['BAC']
 ['BIIB']
 ['BK']
 ['BLK']
 ['BMY']
 ['C']
 ['CAT']
 ['CELG']
 ['CL']
 ['KO']
 ['LLY']
 ['LMT']
 ['MO']
 ['MON']
 ['MRK']
 ['PEP']
 ['PFE']
 ['PG']
 ['PM']
 ['PYPL']
 ['QCOM']
 ['RTN']
 ['RY.TO']
 ['SHOP.TO']
 ['T.TO']
 ['TD.TO']
 ['TXN']
 ['UNH']
 ['UNP']
 ['UPS']
 ['USB']
 ['YCBD']]


In [20]:
marketTicker = yf.Ticker("^GSPC")
marketHist = marketTicker.history(start=start_date, end=end_date)

beta = {}

def beta_volatility(ticker):
    prices = pd.DataFrame(tickers_hist[ticker]["Close"])
    prices.columns = [ticker]
    prices["Market"] = marketHist['Close']

    monthly_change = prices.resample("M").ffill().pct_change()
    monthly_change = monthly_change.drop(index=monthly_change.index[0])

    return (monthly_change.cov()/monthly_change["Market"].var()).iat[0, 1]


for ticker in tickers:
    beta[ticker] = beta_volatility(ticker)

sorted(beta.items(), key=lambda x:x[1], reverse=True)


[('SHOP.TO', 2.7475907339436922),
 ('PYPL', 2.193305236670926),
 ('BA', 1.7387713711598616),
 ('CAT', 1.6534184714952074),
 ('UPS', 1.5430320889311957),
 ('BLK', 1.3281095376598537),
 ('ABT', 1.3104263068093212),
 ('USB', 1.2971088786584422),
 ('QCOM', 1.2793977558010605),
 ('AMZN', 1.2636591614049615),
 ('PM', 1.1651537550803623),
 ('TXN', 1.158627139701168),
 ('PG', 1.0755680237009173),
 ('UNP', 0.9635634449080173),
 ('AXP', 0.8367512182241356),
 ('ACN', 0.7941966209662137),
 ('PEP', 0.7896180076397445),
 ('KO', 0.7773029185547554),
 ('BAC', 0.7608869979450263),
 ('LLY', 0.7371024835053319),
 ('BMY', 0.6469187132974675),
 ('TD.TO', 0.6268886354981764),
 ('RY.TO', 0.6013089816327523),
 ('C', 0.5797286138979415),
 ('BK', 0.5535650947784039),
 ('PFE', 0.539162016045789),
 ('MO', 0.48960658835810733),
 ('CL', 0.48704390419199417),
 ('BIIB', 0.4641406400885526),
 ('MRK', 0.4529446738198995),
 ('T.TO', 0.2948843491627961),
 ('ABBV', 0.21542835233673022),
 ('LMT', 0.16324632397902603),
 ('A

In [ ]:
def correlation(stock1, stock2):
    stock1Hist = tickers_hist[stock1].copy()
    stock2Hist = tickers_hist[stock2].copy()

    prices = pd.DataFrame(stock1Hist["Close"])
    prices.columns = [stock1]
    prices[stock2] = stock2Hist["Close"]
    
    monthly_returns = prices.resample('M').ffill().pct_change()
    monthly_returns.drop(index=monthly_returns.index[0], inplace=True)

    return monthly_returns.corr()

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.